In [1]:
pip install pydocs

  Using cached pydocs-0.2.tar.gz (977 bytes)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
Note: you may need to restart the kernel to use updated packages.


  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> [8 lines of output]
      running egg_info
      creating C:\Users\tejas\AppData\Local\Temp\pip-pip-egg-info-hkzyg1ce\pydocs.egg-info
      writing C:\Users\tejas\AppData\Local\Temp\pip-pip-egg-info-hkzyg1ce\pydocs.egg-info\PKG-INFO
      writing dependency_links to C:\Users\tejas\AppData\Local\Temp\pip-pip-egg-info-hkzyg1ce\pydocs.egg-info\dependency_links.txt
      writing requirements to C:\Users\tejas\AppData\Local\Temp\pip-pip-egg-info-hkzyg1ce\pydocs.egg-info\requires.txt
      writing top-level names to C:\Users\tejas\AppData\Local\Temp\pip-pip-egg-info-hkzyg1ce\pydocs.egg-info\top_level.txt
      writing manifest file 'C:\Users\tejas\AppData\Local\Temp\pip-pip-egg-info-hkzyg1ce\pydocs.egg-info\SOURCES.txt'
      error: package directory 'pydocs' does not exist
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem 

In [2]:
import pandas as pd
from sqlalchemy import create_engine
from binance.client import Client
from binance import BinanceSocketManager

In [3]:
client=Client()

In [4]:
bsm=BinanceSocketManager(client)

In [11]:
socket=bsm.kline_socket('BTCUSDT')  
await socket.__aenter__()
msg=await socket.recv()
print(msg)
await socket.__aexit__(None, None, None)


{'e': 'kline', 'E': 1731776491069, 's': 'BTCUSDT', 'k': {'t': 1731776460000, 'T': 1731776519999, 's': 'BTCUSDT', 'i': '1m', 'f': 4090261202, 'L': 4090263898, 'o': '90572.00000000', 'c': '90536.23000000', 'h': '90575.62000000', 'l': '90536.23000000', 'v': '4.35848000', 'n': 2697, 'x': False, 'q': '394707.83831820', 'V': '0.62469000', 'Q': '56571.46562130', 'B': '0'}}


In [ ]:
def datatransform(msg):
    df=pd.DataFrame({'Time':msg['E'],'Price':msg['k']['c']},index=[0])
    df.Price=df.Price.astype(float)
    df.Time=pd.to_datetime(df.Time,unit='ms')
    return df

In [13]:
datatransform(msg)

,Time,Price
0,2024-11-16 17:01:31.069,90536.23


In [14]:
from sqlalchemy.engine import URL
connection_string = "DRIVER={ODBC Driver 17 for SQL Server};SERVER=dagger;DATABASE=BTCUSDTstream.db;UID=root;PWD=Minute@0514!"
connection_url = URL.create("mssql+pyodbc", query={"odbc_connect": connection_string})

In [16]:
engine=create_engine(connection_url)




ModuleNotFoundError: No module named 'pyodbc'

In [26]:
current_event=pd.Series(pd.to_datetime(0))
while True:
    await socket.__aenter__()
    msg= await socket.recv()
    await socket.__aexit__(None, None, None)
    df=datatransform(msg)
    if df.Time.values>current_event.values:
        current_event=df.Time
        df.to_sql('BTCUSDT',connection, if_exists='append',index=False)


C:\Users\tejas\AppData\Local\Temp\ipykernel_15764\38349914.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('BTCUSDT',connection, if_exists='append',index=False)
C:\Users\tejas\AppData\Local\Temp\ipykernel_15764\38349914.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('BTCUSDT',connection, if_exists='append',index=False)
C:\Users\tejas\AppData\Local\Temp\ipykernel_15764\38349914.py:9: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df.to_sql('BTCUSDT',connection, if_exists='append',index=False)
C:\U

KeyError: 'E'

In [1]:
from binance import Client, ThreadedWebsocketManager, ThreadedDepthCacheManager
client = Client(api_key, api_secret)

# get market depth
depth = client.get_order_book(symbol='BNBBTC')

# place a test market buy order, to place an actual order use the create_order function
order = client.create_test_order(
    symbol='BNBBTC',
    side=Client.SIDE_BUY,
    type=Client.ORDER_TYPE_MARKET,
    quantity=100)

# get all symbol prices
prices = client.get_all_tickers()

# withdraw 100 ETH
# check docs for assumptions around withdrawals
from binance.exceptions import BinanceAPIException
try:
    result = client.withdraw(
        asset='ETH',
        address='<eth_address>',
        amount=100)
except BinanceAPIException as e:
    print(e)
else:
    print("Success")

# fetch list of withdrawals
withdraws = client.get_withdraw_history()

# fetch list of ETH withdrawals
eth_withdraws = client.get_withdraw_history(coin='ETH')

# get a deposit address for BTC
address = client.get_deposit_address(coin='BTC')

# get historical kline data from any date range

# fetch 1 minute klines for the last day up until now
klines = client.get_historical_klines("BNBBTC", Client.KLINE_INTERVAL_1MINUTE, "1 day ago UTC")

# fetch 30 minute klines for the last month of 2017
klines = client.get_historical_klines("ETHBTC", Client.KLINE_INTERVAL_30MINUTE, "1 Dec, 2017", "1 Jan, 2018")

# fetch weekly klines since it listed
klines = client.get_historical_klines("NEOBTC", Client.KLINE_INTERVAL_1WEEK, "1 Jan, 2017")

# socket manager using threads
twm = ThreadedWebsocketManager()
twm.start()

# depth cache manager using threads
dcm = ThreadedDepthCacheManager()
dcm.start()

def handle_socket_message(msg):
    print(f"message type: {msg['e']}")
    print(msg)

def handle_dcm_message(depth_cache):
    print(f"symbol {depth_cache.symbol}")
    print("top 5 bids")
    print(depth_cache.get_bids()[:5])
    print("top 5 asks")
    print(depth_cache.get_asks()[:5])
    print("last update time {}".format(depth_cache.update_time))

twm.start_kline_socket(callback=handle_socket_message, symbol='BNBBTC')

dcm.start_depth_cache(callback=handle_dcm_message, symbol='ETHBTC')

# replace with a current options symbol
options_symbol = 'BTC-241227-41000-C'
dcm.start_options_depth_cache(callback=handle_dcm_message, symbol=options_symbol)

# join the threaded managers to the main thread
twm.join()
dcm.join()

NameError: name 'api_key' is not defined